In [42]:
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='browser'
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import scipy as sp
import scipy.cluster
import shap

In [59]:
deposit_data = pd.read_csv("../../Curated_data/final_dataset.csv", low_memory=False)


In [60]:
total_columns = ['CP_Total','PO_Total', 'PY_Total']

#all covariates
covariates = total_columns[:3] + ['RQD_Pct', 'Cr_ppm', 'Random_Values'] 

In [61]:
# Assuming deposit_data is your DataFrame
# Extract the names of the first 98 columns
phi_columns = deposit_data.columns[10:-1].tolist()

# Display the list of column names
print(phi_columns[-1])

phi_6401


In [62]:
deposit_data['Random_Values'] = np.random.rand(len(deposit_data))

# Deepkriging covariates

In [14]:

phi_columns = deposit_data.columns[10:-1].tolist()

# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

# Assuming deposit_data, covariates, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_rmse_list = []
test_mae_list = []
test_r2_list = []



# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train = train_data[phi_columns + covariates].values
    y_train = train_data['Density_gcm3'].values

    x_test = test_data[phi_columns + covariates].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = nn.Sequential(
        nn.Linear(in_features=p, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5) ,
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=1))


    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))



    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")


Metrics for Fold 1:
Metrics for Test set:
  MSE: 0.0033
  RMSE: 0.0572
  MAE: 0.0392
  R^2: 0.7952


Metrics for Fold 2:
Metrics for Test set:
  MSE: 0.0028
  RMSE: 0.0528
  MAE: 0.0353
  R^2: 0.7874


Metrics for Fold 3:
Metrics for Test set:
  MSE: 0.0035
  RMSE: 0.0595
  MAE: 0.0393
  R^2: 0.7599


Metrics for Fold 4:
Metrics for Test set:
  MSE: 0.0026
  RMSE: 0.0509
  MAE: 0.0362
  R^2: 0.8084


Metrics for Fold 5:
Metrics for Test set:
  MSE: 0.0044
  RMSE: 0.0666
  MAE: 0.0437
  R^2: 0.7537


Metrics for Fold 6:
Metrics for Test set:
  MSE: 0.0028
  RMSE: 0.0525
  MAE: 0.0364
  R^2: 0.7837


Metrics for Fold 7:
Metrics for Test set:
  MSE: 0.0027
  RMSE: 0.0516
  MAE: 0.0394
  R^2: 0.8522


Metrics for Fold 8:
Metrics for Test set:
  MSE: 0.0028
  RMSE: 0.0530
  MAE: 0.0370
  R^2: 0.8438


Metrics for Fold 9:
Metrics for Test set:
  MSE: 0.0048
  RMSE: 0.0695
  MAE: 0.0468
  R^2: 0.6194


Metrics for Fold 10:
Metrics for Test set:
  MSE: 0.0038
  RMSE: 0.0618
  MAE: 0.0431
  R^

# NN Covariates

In [48]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

phi_columns = ['X','Y','Z']
p = len(covariates) + len(phi_columns)

# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")



# Assuming deposit_data, covariates, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_rmse_list = []
test_mae_list = []
test_r2_list = []



# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train = train_data[phi_columns + covariates].values
    y_train = train_data['Density_gcm3'].values

    x_test = test_data[phi_columns + covariates].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network
    model = nn.Sequential(
        nn.Linear(in_features=p, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5) ,
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=100, out_features=100),
        nn.ReLU(),
        nn.BatchNorm1d(100),
        nn.Linear(in_features=100, out_features=1))


    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))



    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")


Metrics for Fold 1:
Metrics for Test set:
  MSE: 0.0122
  RMSE: 0.1107
  MAE: 0.0708
  R^2: 0.2327


Metrics for Fold 2:
Metrics for Test set:
  MSE: 0.0111
  RMSE: 0.1052
  MAE: 0.0660
  R^2: 0.1565


Metrics for Fold 3:
Metrics for Test set:
  MSE: 0.0125
  RMSE: 0.1119
  MAE: 0.0692
  R^2: 0.1509


Metrics for Fold 4:
Metrics for Test set:
  MSE: 0.0124
  RMSE: 0.1113
  MAE: 0.0710
  R^2: 0.0819


Metrics for Fold 5:
Metrics for Test set:
  MSE: 0.0155
  RMSE: 0.1244
  MAE: 0.0788
  R^2: 0.1389


Metrics for Fold 6:
Metrics for Test set:
  MSE: 0.0109
  RMSE: 0.1045
  MAE: 0.0697
  R^2: 0.1414


Metrics for Fold 7:
Metrics for Test set:
  MSE: 0.0161
  RMSE: 0.1267
  MAE: 0.0776
  R^2: 0.1107


Metrics for Fold 8:
Metrics for Test set:
  MSE: 0.0163
  RMSE: 0.1278
  MAE: 0.0785
  R^2: 0.0935


Metrics for Fold 9:
Metrics for Test set:
  MSE: 0.0112
  RMSE: 0.1060
  MAE: 0.0723
  R^2: 0.1140


Metrics for Fold 10:
Metrics for Test set:
  MSE: 0.0183
  RMSE: 0.1354
  MAE: 0.0881
  R^

In [51]:
covariates

['CP_Total', 'PO_Total', 'PY_Total', 'RQD_Pct', 'Cr_ppm', 'Random_Values']

# Reg kriging covariates

In [128]:

import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from pykrige.ok import OrdinaryKriging

np.random.seed(42)

y = deposit_data['Density_gcm3'].values[:, np.newaxis]  # Keep variable as the output
x = deposit_data[['X', 'Y', 'Z','CP_Total', 'PO_Total', 'PY_Total', 'RQD_Pct', 'Cr_ppm', 'Random_Values']].values
x = x.reshape(len(deposit_data), 9)

num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

mse_list, mae_list, r2_list = [], [], []
for train_index, test_index in kf.split(x):
    X_cv_train, X_cv_test = x[train_index], x[test_index]
    y_cv_train, y_cv_test = y[train_index], y[test_index]

    # Fit a linear regression model
    regression_model = LinearRegression()
    regression_model.fit(X_cv_train, y_cv_train)

    # Predictions from the regression model
    y_cv_pred = regression_model.predict(X_cv_test)

    # Residuals (the difference between actual and predicted values)
    residuals = y_cv_test - y_cv_pred

    # Ordinary Kriging on residuals
    #ok = OrdinaryKriging(X_cv_test[:, 0], X_cv_test[:, 1], y_cv_test, variogram_model='linear', verbose=False)
    # kriging_pred, _ = ok.execute('grid', X_cv_test[:, 0], X_cv_test[:, 1])
    ok = OrdinaryKriging(X_cv_test[:, 0], X_cv_test[:, 1], residuals, variogram_model='linear', verbose=False)
    kriging_pred, _ = ok.execute('grid', X_cv_test[:, 0], X_cv_test[:, 1])


    # Use Kriging predictions directly
    final_predictions = kriging_pred

    # Combine regression predictions with kriging predictions
    final_cv_predictions = y_cv_pred + kriging_pred

    # Calculate and store metrics
    mse = np.mean((y_cv_test - final_cv_predictions) ** 2)
    mae = np.mean(np.abs(y_cv_test - final_cv_predictions))
    sst = np.mean((y_cv_test - np.mean(y_cv_test)) ** 2)*len(y_cv_test)
    ssr = np.mean((final_cv_predictions - y_cv_test) ** 2)*len(y_cv_test)
    r2 = 1 - (ssr / sst)


    mse_list.append(mse)
    mae_list.append(mae)
    r2_list.append(r2)

# Calculate mean metrics across folds
mean_mse = np.mean(mse_list)
mean_mae = np.mean(mae_list)
mean_r2 = np.mean(r2_list)

# Print mean metrics
print(f"Mean Squared Error (MSE): {mean_mse}")
print(f"Mean Absolute Error (MAE): {mean_mae}")
print(f"Mean R-squared (R2): {mean_r2}")

Mean Squared Error (MSE): 0.013664941227511046
Mean Absolute Error (MAE): 0.07514578640239988
Mean R-squared (R2): 0.12803822086217215


# Deepkriging no covariates

In [65]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

torch.manual_seed(42)
np.random.seed(42)

p = len(covariates) + len(phi_columns)
phi_columns = deposit_data.columns[10:-1].tolist()


# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

# Assuming deposit_data, phi_columns, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_mae_list = []
test_r2_list = []

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define a neural network without covariates
class GroupedNet(nn.Module):
    def __init__(self, input_size_phi, output_size):
        super(GroupedNet, self).__init__()
        self.phi_branch = nn.Sequential(
            nn.Linear(in_features=input_size_phi, out_features=1)
        )
        
        self.combine_layer = nn.Sequential(
            nn.Linear(1, 100), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=1)
        )

    def forward(self, input_phi):
        phi_output = self.phi_branch(input_phi)
        x = phi_output
        output = self.combine_layer(x)
        return output

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train_phi = train_data[phi_columns].values
    y_train = train_data['Density_gcm3'].values

    x_test_phi = test_data[phi_columns].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network without covariates
    model = GroupedNet(input_size_phi=len(phi_columns), output_size=1)

    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train_phi, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test_phi, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test_phi, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")



Metrics for Fold 1:
Metrics for Test set:
  MSE: 0.0028
  RMSE: 0.0528
  MAE: 0.0379
  R^2: 0.8256


Metrics for Fold 2:
Metrics for Test set:
  MSE: 0.0062
  RMSE: 0.0788
  MAE: 0.0539
  R^2: 0.5258


Metrics for Fold 3:
Metrics for Test set:
  MSE: 0.0040
  RMSE: 0.0634
  MAE: 0.0424
  R^2: 0.7270


Metrics for Fold 4:
Metrics for Test set:
  MSE: 0.0031
  RMSE: 0.0558
  MAE: 0.0406
  R^2: 0.7691


Metrics for Fold 5:
Metrics for Test set:
  MSE: 0.0036
  RMSE: 0.0602
  MAE: 0.0448
  R^2: 0.7987


Metrics for Fold 6:
Metrics for Test set:
  MSE: 0.0030
  RMSE: 0.0550
  MAE: 0.0405
  R^2: 0.7618


Metrics for Fold 7:
Metrics for Test set:
  MSE: 0.0041
  RMSE: 0.0643
  MAE: 0.0470
  R^2: 0.7708


Metrics for Fold 8:
Metrics for Test set:
  MSE: 0.0045
  RMSE: 0.0670
  MAE: 0.0487
  R^2: 0.7512


Metrics for Fold 9:
Metrics for Test set:
  MSE: 0.0048
  RMSE: 0.0692
  MAE: 0.0472
  R^2: 0.6220


Metrics for Fold 10:
Metrics for Test set:
  MSE: 0.0033
  RMSE: 0.0576
  MAE: 0.0438
  R^

# NN no covariates

In [66]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

phi_columns = ['X','Y','Z']
p = len(covariates) + len(phi_columns)
torch.manual_seed(42)
np.random.seed(42)


# Function to print evaluation metrics
def print_metrics(actual, predicted, set_name):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)

    print(f"Metrics for {set_name} set:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R^2: {r2:.4f}\n")

# Assuming deposit_data, phi_columns, and other necessary variables are defined

# Create an array to store metrics for each fold
test_mse_list = []
test_mae_list = []
test_r2_list = []

# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define a neural network without covariates
class GroupedNet(nn.Module):
    def __init__(self, input_size_phi, output_size):
        super(GroupedNet, self).__init__()
        self.phi_branch = nn.Sequential(
            nn.Linear(in_features=input_size_phi, out_features=1)
        )
        
        self.combine_layer = nn.Sequential(
            nn.Linear(1, 100), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=100, out_features=100),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            nn.Linear(in_features=100, out_features=1)
        )

    def forward(self, input_phi):
        phi_output = self.phi_branch(input_phi)
        x = phi_output
        output = self.combine_layer(x)
        return output

# Perform k-fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(deposit_data)):
    train_data, test_data = deposit_data.iloc[train_index], deposit_data.iloc[test_index]

    x_train_phi = train_data[phi_columns].values
    y_train = train_data['Density_gcm3'].values

    x_test_phi = test_data[phi_columns].values
    y_test = test_data['Density_gcm3'].values

    # Define your neural network without covariates
    model = GroupedNet(input_size_phi=len(phi_columns), output_size=1)

    mse_loss = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses = []  # To store training losses during training
    test_losses = []   # To store test losses during training

    # Training loop
    for step in range(601):
        pre = model(torch.tensor(x_train_phi, dtype=torch.float32))
        mse = mse_loss(pre, torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32))
        cost = mse

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        pre_test = model(torch.tensor(x_test_phi, dtype=torch.float32))
        mse_test = mse_loss(pre_test, torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32))
        test_losses.append(mse_test.item())

    # Store metrics for this fold
    test_predictions_fold = model(torch.tensor(x_test_phi, dtype=torch.float32)).detach().numpy().flatten()
    test_mse_list.append(mean_squared_error(y_test, test_predictions_fold))
    test_mae_list.append(mean_absolute_error(y_test, test_predictions_fold))
    test_r2_list.append(r2_score(y_test, test_predictions_fold))

    # Print metrics for the current fold
    print(f"\nMetrics for Fold {fold + 1}:")
    print_metrics(y_test, test_predictions_fold, "Test")

# Print average metrics across folds
print("\nAverage Metrics Across Folds:")
print(f"  Average MSE: {np.mean(test_mse_list):.4f}")
print(f"  Average MAE: {np.mean(test_mae_list):.4f}")
print(f"  Average R2: {np.mean(test_r2_list):.4f}")
print(f"  STD MSE: {np.std(test_mse_list):.4f}")
print(f"  STD MAE: {np.std(test_mae_list):.4f}")
print(f"  STD R2: {np.std(test_r2_list):.4f}")



Metrics for Fold 1:
Metrics for Test set:
  MSE: 0.0142
  RMSE: 0.1191
  MAE: 0.0762
  R^2: 0.1108


Metrics for Fold 2:
Metrics for Test set:
  MSE: 0.0134
  RMSE: 0.1156
  MAE: 0.0771
  R^2: -0.0197


Metrics for Fold 3:
Metrics for Test set:
  MSE: 0.0147
  RMSE: 0.1211
  MAE: 0.0784
  R^2: 0.0049


Metrics for Fold 4:
Metrics for Test set:
  MSE: 0.0131
  RMSE: 0.1143
  MAE: 0.0756
  R^2: 0.0319


Metrics for Fold 5:
Metrics for Test set:
  MSE: 0.0181
  RMSE: 0.1344
  MAE: 0.0844
  R^2: -0.0050


Metrics for Fold 6:
Metrics for Test set:
  MSE: 0.0124
  RMSE: 0.1114
  MAE: 0.0744
  R^2: 0.0239


Metrics for Fold 7:
Metrics for Test set:
  MSE: 0.0184
  RMSE: 0.1358
  MAE: 0.0853
  R^2: -0.0210


Metrics for Fold 8:
Metrics for Test set:
  MSE: 0.0180
  RMSE: 0.1341
  MAE: 0.0846
  R^2: 0.0012


Metrics for Fold 9:
Metrics for Test set:
  MSE: 0.0126
  RMSE: 0.1124
  MAE: 0.0733
  R^2: 0.0039


Metrics for Fold 10:
Metrics for Test set:
  MSE: 0.0177
  RMSE: 0.1330
  MAE: 0.0824
 

# Regression kriging no covariates

In [124]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pykrige.ok import OrdinaryKriging
np.random.seed(42)


y = deposit_data['Density_gcm3'].values[:, np.newaxis]  # Keep variable as the output
x = deposit_data[['X', 'Y', 'Z']].values
x = x.reshape(len(deposit_data), 3)

num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

mse_list, mae_list, r2_list = [], [], []
for train_index, test_index in kf.split(x):
    X_cv_train, X_cv_test = x[train_index], x[test_index]
    y_cv_train, y_cv_test = y[train_index], y[test_index]

    # Fit a linear regression model
    regression_model = LinearRegression()
    regression_model.fit(X_cv_train, y_cv_train)

    # Predictions from the regression model
    y_cv_pred = regression_model.predict(X_cv_test)

    # Residuals (the difference between actual and predicted values)
    residuals = y_cv_test - y_cv_pred

    # Ordinary Kriging on residuals
    ok = OrdinaryKriging(X_cv_test[:, 0], X_cv_test[:, 1], y_cv_test, variogram_model='linear', verbose=False)
    kriging_pred, _ = ok.execute('grid', X_cv_test[:, 0], X_cv_test[:, 1])


    # Use Kriging predictions directly
    final_predictions = kriging_pred

    # Combine regression predictions with kriging predictions
    final_cv_predictions = y_cv_pred + kriging_pred
    #final_cv_predictions =  kriging_pred

    # Calculate and store metrics
    mse = np.mean((y_cv_test - final_cv_predictions) ** 2)
    mae = np.mean(np.abs(y_cv_test - final_cv_predictions))
    sst = np.mean((y_cv_test - np.mean(y_cv_test)) ** 2)*len(y_cv_test)
    ssr = np.mean((final_cv_predictions - y_cv_test) ** 2)*len(y_cv_test)
    r2 = 1 - (ssr / sst)


    mse_list.append(mse)
    mae_list.append(mae)
    r2_list.append(r2)

# Calculate mean metrics across folds
mean_mse = np.mean(mse_list)
mean_mae = np.mean(mae_list)
mean_r2 = np.mean(r2_list)

# Print mean metrics
print(f"Mean Squared Error (MSE): {mean_mse}")
print(f"Mean Absolute Error (MAE): {mean_mae}")
print(f"Mean R-squared (R2): {mean_r2}")

Mean Squared Error (MSE): 0.22492727759675396
Mean Absolute Error (MAE): 0.4598115759529028
Mean R-squared (R2): -13.745761019578921
